In [ ]:
import numpy as np
import scipy as sp
from pathlib import Path
import pandas as pd

# Data loading

In [116]:
# Load all .mat files and extract relevant data
path = Path('.').glob('*.mat')
all_subjects = [sp.io.loadmat(f, squeeze_me=True)['subjectData'] for f in path]
columns = all_subjects[0].dtype.names
print("Columns: ", columns)
print("Loaded data for", len(all_subjects), "subjects.")

Columns:  ('INIT_DELAY', 'MI_DURATION', 'subjectId', 'fs', 'trialsData', 'trialsLabels')
Loaded data for 6 subjects.


In [117]:
# Assert that certain fields are consistent across all subjects in order to discard them
for field in ['INIT_DELAY', 'MI_DURATION', 'fs']:
    assert all(s[field] == all_subjects[0][field] for s in all_subjects)

fs = all_subjects[0]['fs']
init_delay = all_subjects[0]['INIT_DELAY']
mi_duration = all_subjects[0]['MI_DURATION']
print("Common parameters across all subjects:")
print("Sampling frequency: "+ str(fs)+" Hz")
print("Initial delay: "+ str(init_delay)+" s")
print("Motor imagery duration: "+ str(mi_duration)+" s")

Common parameters across all subjects:
Sampling frequency: 256 Hz
Initial delay: 0 s
Motor imagery duration: 6 s


In [143]:
# Extract EEG data and corresponding labels (1 for left hand, 2 for right hand)
X = np.array([s['trialsData'] for s in all_subjects])
y = np.array([s['trialsLabels'] for s in all_subjects])


# print("Data shape (subjects, trials, channels, timepoints): ", X.shape)
print(type(X[0]))

<class 'numpy.ndarray'>
